In [1]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

2019-04-26 17:58:33.675000


In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt

In [4]:
import statsmodels.api as sm

In [5]:
df = sm.datasets.anes96.load_pandas().data

In [6]:
from __future__ import division
import pandas as pd ; pd.set_option("display.max_columns",200)
import pickle
import numpy as np
import string
import sys

In [7]:
import sys
sys.path.append(".\Income_tax_function")
import IRPP_from_scratch as ir

In [8]:
store_path = '../Data/hdf/edp_2015_final.h5'
Store = pd.HDFStore(store_path)

In [9]:
Store

<class 'pandas.io.pytables.HDFStore'>
File path: ../Data/hdf/edp_2015_final.h5
/DECES                         frame        (shape->[358338,19])  
/DESCENDANCE                   frame        (shape->[1320391,41]) 
/EAR2004_FAMILLE               frame        (shape->[48316,13])   
/EAR2004_INDIVIDU              frame        (shape->[277933,86])  
/EAR2004_LOGEMENT              frame        (shape->[89892,52])   
/EAR2005_FAMILLE               frame        (shape->[78041,13])   
/EAR2005_INDIVIDU              frame        (shape->[283781,86])  
/EAR2005_LOGEMENT              frame        (shape->[91733,52])   
/EAR2006_FAMILLE               frame        (shape->[78227,13])   
/EAR2006_INDIVIDU              frame        (shape->[287786,86])  
/EAR2006_LOGEMENT              frame        (shape->[92761,52])   
/EAR2007_FAMILLE               frame        (shape->[78230,13])   
/EAR2007_INDIVIDU              frame        (shape->[283228,86])  
/EAR2007_LOGEMENT              frame        (shape

In [10]:
def load_fisc_i_by_year(year = 2014):
    df_fisci = pd.read_hdf(store_path, 'FISC_INDIVIDU_{}'.format(year))
    return df_fisci
def load_fisc_r_by_year(year = 2014):
    df_fiscr = pd.read_hdf(store_path, 'FISC_REVENU_{}'.format(year))
    return df_fiscr
def load_fisc_l_by_year(year = 2014):
    df_fiscr = pd.read_hdf(store_path, 'FISC_LOGEMENT_{}'.format(year))
    return df_fiscr

def load_df_fiscrevdet_by_year(year = 2014):
    df_fiscrevdet = pd.read_hdf(store_path, 'FISC_REVDET_{}'.format(year))
    return df_fiscrevdet
    

In [11]:
# %%time
# df_ear_dict = dict()
# for year in range(2014, 2016):
#     df_ear_temp = pd.read_hdf(store_path, 'EAR{}_INDIVIDU'.format(year))
#     df_ear_temp["AN_EDP"] = year
#     df_ear_dict[year] = df_ear_temp


In [12]:
import datetime

In [13]:
%%time
df_ear= pd.read_hdf(store_path, 'EAR{}_INDIVIDU'.format(2015))
df_ear["AN_EDP"] = 2015
for year in range(2010, 2015):
    print year
    df_ear_temp = pd.read_hdf(store_path, 'EAR{}_INDIVIDU'.format(year))
    df_ear_temp["AN_EDP"] = year
    df_ear = pd.concat([df_ear,df_ear_temp])
    print year, datetime.datetime.now()
del df_ear_temp #1m 22 s avec ear range(2013,2015)

2010
2010 2019-04-26 17:59:13.558000
2011
2011 2019-04-26 17:59:40.223000
2012
2012 2019-04-26 18:00:08.652000
2013
2013 2019-04-26 18:00:42.053000
2014
2014 2019-04-26 18:01:20.498000
Wall time: 2min 42s


In [14]:
#df_ear.sort("ID_LOG_DIFF").head()

1. Sort by year  
2. Find last ID_DIFF for a given year  
3. take ID_DIFF ID_LOG 


1. Sort by year 

In [15]:
%%time
df_ear.sort("AN_EDP", inplace=True) #25s

Wall time: 17.3 s


<span>2. Find last ID_DIFF for a given year  

In [16]:
%%time
last_id_diff = df_ear.groupby('ID_DIFF')[["ID_DIFF", "AN_EDP", "ID_LOG_DIFF"]].last()

Wall time: 2.09 s


In [17]:
%%time
merge = pd.merge(df_ear, last_id_diff, on = ["ID_LOG_DIFF", "AN_EDP"], how = 'inner', suffixes=("", "_to_drop")) #18s

Wall time: 18.7 s


In [18]:
merge["ID_DIFF_Filled"] = merge.ID_DIFF_to_drop

On a gardé les informations les plus fraiches sur un logement d'un individu particulier.

<span> 3. On repli les id_diff pour un logement particulier

In [19]:
merge.ID_DIFF_Filled.head(10)

0    2542781
1    2542781
2     469663
3     469663
4     469663
5     970783
6     970783
7    3000998
8    3000998
9    3256490
Name: ID_DIFF_Filled, dtype: float64

Je renome les variables pour un merge avec df_fisci.

In [20]:
merge["ANAIS"] = merge.ANAI
merge['JNAIS'] = merge.JNAI
merge['MNAIS']= merge.MNAI

In [21]:
%%time
biologic_path = (u"./Pickle/biologic/biologic_households{}.p".format(2013))
biologic_2013 = pickle.load( 
        open(biologic_path, 'rb'))
fisci = load_fisc_i_by_year(2013)
df_fisci = fisci[fisci.ID_FISC_LOG_DIFF.isin(biologic_2013)]
id_diff_fisci = set(df_fisci[df_fisci.ID_DIFF.notnull()].ID_DIFF)

Wall time: 19.3 s


In [22]:
%%time
last_id_diff_fisci = df_fisci.groupby('ID_DIFF')[["ID_DIFF", "AN_FISC", "ID_FISC_LOG_DIFF"]].last()

Wall time: 443 ms


In [23]:
%%time
merge_fisci = pd.merge(df_fisci, last_id_diff_fisci, on = ["ID_FISC_LOG_DIFF", "AN_FISC"], how = 'inner', suffixes=("", "_to_drop"))#1s

Wall time: 491 ms


In [24]:
merge_fisci["ID_DIFF_Filled"] = merge_fisci.ID_DIFF_to_drop

In [25]:
# %%time
# merge_fisci.sort('ID_DIFF_Filled', inplace = True) #Peut être supprimé pour gagner du temps
# merge.sort('ID_DIFF_Filled', inplace = True) #Peut être supprimé pour gagner du temps

### On Merge fisci et les ear sur ID_DIFF_FILLED et les dates de naissances

In [26]:
%%time
merge_ear_fisci = pd.merge(merge_fisci, merge, on=["ID_DIFF_Filled","ANAIS", "JNAIS","MNAIS"], suffixes = ('', '_to_drop'), how = "inner")#1m24s

Wall time: 2min 49s


In [27]:
print "A"

A


In [28]:
#merge_ear_fisci.sort('ID_FISC_LOG_DIFF').head()

In [29]:
merge_ear_fisci.ID_FISC_LOG_DIFF.nunique() #22384 logements matché avec ear 2013,2014,2015 ; 36954 avec 2010,2015

37428

In [30]:
merge_ear_fisci.reset_index(inplace = True)

In [31]:
merge_ear_fisci.drop("ID_DIFF_to_drop", axis = 1, inplace = True)

In [32]:
merge_ear_fisci.to_hdf(
    "./Generated_hdf/FSCI_EAR_BIO.h5",  
    "ear_2010_2015_fisci_bio_2013")

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\pandas\io\pytables.py:2577: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->['ORDREFIP', 'TYPE_FISC', 'SEXE', 'CODNAIS', 'CIDECI', 'ZOXYZD', 'ZOXYZP', 'T_CHARGE', 'CNCOIPV', 'CSDEP', 'ANAR', 'APAF', 'ARRIV', 'CIL', 'COUPLE', 'CPOP', 'CRP_C_C', 'CRP_C_R', 'CS', 'DCETUF', 'DCLT', 'DCNAI', 'DCRAN', 'DEPCOM_CODE', 'DIPL', 'DISTIL_CODE', 'EMPL', 'ETUD', 'FIL', 'ILETUD', 'ILTD', 'IMMI', 'INAT', 'INATCC', 'IPRM', 'IRAND', 'IRIS_ILOT', 'LIENF', 'LPRF', 'LPRM', 'MOCO', 'MODV', 'NAF', 'NATE', 'NATE_C_C', 'NATN', 'NATN_C_C', 'NUMF', 'PNAI', 'PNAI_C_C', 'POSP', 'PRAN', 'PRAN_C_C', 'PROF', 'RECH', 'REGNAI', 'REGRAN', 'TP', 'TYPACT']]

  warnings.warn(ws, PerformanceWarning)


In [33]:
merge_ear_fisci = pd.read_hdf(
     "./Generated_hdf/FSCI_EAR_BIO.h5",  
    "ear_2010_2015_fisci_bio_2013")

In [34]:
merge_ear_fisci.REG_CODE.head()

0    73
1    73
2    72
3    72
4    82
Name: REG_CODE, dtype: int64

In [35]:
merge_ear_fisci["Diplome_homme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 1].DIPL_X
merge_ear_fisci["Diplome_femme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 2].DIPL_X
merge_ear_fisci["Categorie_sociale_homme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 1].CS
merge_ear_fisci["Categorie_sociale_femme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 2].CS
merge_ear_fisci["Travaille_dans_commune_ILTD_homme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 1].ILTD
merge_ear_fisci["Travaille_dans_commune_ILTD_femme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 2].ILTD
merge_ear_fisci["Type_contrat_travail_EMPL_homme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 1].EMPL 
merge_ear_fisci["Type_contrat_travail_EMPL_femme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 2].EMPL
merge_ear_fisci["Immigre_IMMI_homme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 1].IMMI
merge_ear_fisci["Immigre_IMMI_femme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 2].IMMI
merge_ear_fisci["Position_professionnelle_homme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 1].POSP
merge_ear_fisci["Position_professionnelle_femme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 2].POSP

merge_ear_fisci["Temps_recherche_emploi_homme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 1].RECH
merge_ear_fisci["Temps_recherche_emploi_femme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 2].RECH 
merge_ear_fisci["Region_homme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 1].REG_CODE
merge_ear_fisci["Region_femme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 2].REG_CODE
merge_ear_fisci["Temps_partiel_homme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 1].TP 
merge_ear_fisci["Temps_partiel_femme"] = merge_ear_fisci[merge_ear_fisci.SEXE == 2].TP


In [36]:
merge_ear_fisci = merge_ear_fisci.drop_duplicates(subset = ["ID_FISC_LOG_DIFF", 'ANAIS', 'JNAIS', 'MNAIS', 'SEXE'] )

In [37]:
(merge_ear_fisci.groupby("ID_FISC_LOG_DIFF").AN_FISC.count()).value_counts()

2    34445
1     2983
dtype: int64

In [38]:
merge_ear_fisci.TC.value_counts(normalize=True)

1    0.791438
2    0.208562
dtype: float64

In [39]:
vars_to_sum = ["Diplome", "Categorie_sociale", 'Travaille_dans_commune_ILTD', 'Type_contrat_travail_EMPL', 'Immigre_IMMI',
'Position_professionnelle', "Temps_recherche_emploi", 'Region', 'Temps_partiel']

In [40]:
vars_to_sum_homme = [var + "_homme" for var in vars_to_sum]
vars_to_sum_femme = [var + "_femme" for var in vars_to_sum]

In [41]:
vars_to_sum_homme

['Diplome_homme',
 'Categorie_sociale_homme',
 'Travaille_dans_commune_ILTD_homme',
 'Type_contrat_travail_EMPL_homme',
 'Immigre_IMMI_homme',
 'Position_professionnelle_homme',
 'Temps_recherche_emploi_homme',
 'Region_homme',
 'Temps_partiel_homme']

Bon ben vu que le groupby ne marche pas (va savoir pourquoi), on fait un merge en deux partie.

In [42]:
df_homme = merge_ear_fisci[vars_to_sum_homme+ ["ID_FISC_LOG_DIFF"]][merge_ear_fisci.SEXE==1]
df_femme = merge_ear_fisci[vars_to_sum_femme+ ["ID_FISC_LOG_DIFF"]][merge_ear_fisci.SEXE==2]

In [43]:
df_homme.set_index("ID_FISC_LOG_DIFF", inplace = True)
df_femme.set_index("ID_FISC_LOG_DIFF", inplace = True)

In [44]:
df_homme_femme = df_homme.combine_first( df_femme)

In [45]:
df_homme_femme.reset_index(inplace = True)

In [46]:
#df_homme_femme.head(2)

####merge pour rajouter la variable TC

In [47]:
df_homme_femme2 = pd.merge(df_homme_femme, merge_ear_fisci[["ID_FISC_LOG_DIFF", "TC"]], on="ID_FISC_LOG_DIFF", how="inner")

In [48]:
df_homme_femme2 = df_homme_femme2.drop_duplicates()

In [49]:
df_homme_femme2.to_hdf(
    "./Generated_hdf/FSCI_EAR_BIO.h5",  
    "ear_groupby_id_fisc_log_diff_homme_femme")

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\pandas\io\pytables.py:2577: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['Categorie_sociale_homme', 'Immigre_IMMI_homme', 'Position_professionnelle_femme', 'Position_professionnelle_homme', 'Temps_partiel_femme', 'Temps_partiel_homme', 'Temps_recherche_emploi_femme', 'Temps_recherche_emploi_homme', 'Travaille_dans_commune_ILTD_femme', 'Travaille_dans_commune_ILTD_homme', 'Type_contrat_travail_EMPL_femme', 'Type_contrat_travail_EMPL_homme']]

  warnings.warn(ws, PerformanceWarning)


In [50]:
import datetime
print datetime.datetime.now()

2019-04-26 18:05:20.080000


In [51]:
stop_time = datetime.datetime.now(); print("now: ",stop_time);
execution_time = stop_time - start_time; print("execution_time: ",execution_time)
stop_time = start_time
#clear  memory
%reset -f

('now: ', datetime.datetime(2019, 4, 26, 18, 5, 20, 94000))
('execution_time: ', datetime.timedelta(0, 406, 419000))
